In [87]:
import pandas as pd
import numpy as np
import math

# Load the datasets
maxz_data = pd.read_csv("C:/Users/akash/OneDrive - Cornell University/Desktop/test/maxz_values.csv")
minz_data = pd.read_csv("C:/Users/akash/OneDrive - Cornell University/Desktop/test/minz_values.csv")
width_df  = pd.read_csv("C:/Users/akash/OneDrive - Cornell University/Desktop/test/W1.csv")
length_df = pd.read_csv("C:/Users/akash/OneDrive - Cornell University/Desktop/test/W2.csv")

# Function to calculate filtered mean
def calculate_filtered_mean(column):
    # Calculate Q1, Q3, and IQR
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter the column to remove outliers
    filtered_values = column[(column >= lower_bound) & (column <= upper_bound)]
    return filtered_values.mean()

# Apply the function to calculate filtered means for maxZ columns
filtered_means = maxz_data.iloc[:, 1:].apply(calculate_filtered_mean)

# Rename the minZ columns to match maxZ for alignment
minz_data_renamed = minz_data.rename(columns=lambda x: x.replace('minZ', 'maxZ') if 'minZ' in x else x)

# Subtract the minZ values from the corresponding maxZ filtered means to calculate heights
filtered_means_dict = filtered_means.to_dict()
minz_data_numeric_renamed = minz_data_renamed.iloc[:, 1:]  # Exclude the timestamp column
height_data = minz_data_numeric_renamed.apply(lambda col: filtered_means_dict[col.name] - col, axis=0)

# Combine the heights with the timestamp column
height_df = pd.concat([minz_data['timestamp'], height_data], axis=1)
height_df.columns = ['timestamp'] + [f'height_{i}' for i in range(height_df.shape[1] - 1)]

In [91]:
area_df = pd.DataFrame()
area_df['timestamp'] = height_df['timestamp']

volume_df = pd.DataFrame()
volume_df['timestamp'] = height_df['timestamp']

# The exponent we use repeatedly
n = 1.6075
for i in range(40):
    # Column names
    height_col = f'height_{i}'
    width_col  = f'W1_{i}'
    length_col = f'W2_{i}'
    area_col   = f'area_{i}'
    volume_col = f'volume_{i}'
    
    # Extract columns (Series) from each DataFrame
    a = height_df[height_col]            # height
    b = width_df[width_col] / 2.0        # width/2
    c = length_df[length_col] / 2.0      # length/2
    
    # Calculate each row's area
    # A = 2π * ( ((a*b)^n + (a*c)^n + (b*c)^n)/3 )^(1/n ) + π * b * c
    term = ((a*b)**n + (a*c)**n + (b*c)**n) / 3.0
    area_df[area_col] = 2.0*math.pi * (term ** (1.0/n)) + math.pi * b * c
    # Calculate each row's volume
    volume_df[volume_col] = (2.0/3.0) * math.pi * a * b * c



height_df.to_csv("height_df.csv", index=False)
area_df.to_csv("area_df.csv", index=False)
volume_df.to_csv("volume_df.csv", index=False)